In [ ]:
!pip3 install tinkoff-investments
!pip3 install yfinance
!pip3 install pandas
!pip3 install numpy
!pip3 install decimal
!pip3 install locale

In [25]:
from tinkoff.invest import Client

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'

from helpers_module import helpers
from datetime import timedelta
from pathlib import Path
from datetime import datetime, timedelta, timezone
from decimal import Decimal
from tinkoff.invest import CandleInterval, Client, HistoricCandle, Quotation, SubscriptionInterval
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)
from tinkoff.invest.services import MarketDataCache
from tinkoff.invest.utils import now

__all__ = (
    "get_intervals",
    "quotation_to_decimal",
    "decimal_to_quotation",
    "candle_interval_to_subscription_interval",
    "now",
    "candle_interval_to_timedelta",
    "ceil_datetime",
    "floor_datetime",
    "dataclass_from_dict",
    "datetime_range_floor",
)


'''MoneyValue — используется для параметров, у которых есть денежный эквивалент. Возьмем для примера стоимость ценных бумаг — тип состоит из трех параметров:
1) currency — строковый ISO-код валюты, например RUB или USD;
2) units — целая часть суммы;
3) nano — дробная часть суммы, миллиардные доли единицы.
'''
# Quotation type = MoneyValue. We need to convert this to decimal in order to fetch price per share
def quotation_to_decimal(quotation: Quotation) -> Decimal:
    fractional = quotation.nano / Decimal("10e8")
    return Decimal(quotation.units) + fractional

def get_final_stock_volumes(_input):
    return f'Объем: {_input:,} ₽'

def get_final_lots(_lots):
    return f'Кол-во лотов: {_lots:,} шт.'


def main():
    with Client(TOKEN) as client:
        settings = MarketDataCacheSettings(base_cache_dir=Path("market_data_cache"))
        market_data_cache = MarketDataCache(settings=settings, services=client)
        for candle in market_data_cache.get_all_candles(
            figi="BBG004731032",
            from_=now() - timedelta(days=30),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            final_stock_volume_rub = int(candle.volume * quotation_to_decimal(candle.close))
            print(f'Лукойл (LKOH), Время: {candle.time}, {get_final_lots(candle.volume)}, {get_final_stock_volumes(final_stock_volume_rub)}, Цена: {quotation_to_decimal(candle.close)} ₽')
            # Лукойл (LKOH), Время: 2023-05-10 07:03:00+00:00, Кол-во лотов: 16,288 шт., Объем: 758,043,520,000 ₽, Цена: 4654.00000000 ₽
            # Лукойл (LKOH), Время: 2023-05-10 07:03:00+00:00, 16288, 75804352, Цена: 4654.00000000 ₽



            

    return 0

if __name__ == "__main__":
    main()


Лукойл (LKOH), Время: 2023-04-10 11:25:00+00:00, Кол-во лотов: 312 шт., Объем: 14,484,600,000 ₽, Цена: 4642.50000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:26:00+00:00, Кол-во лотов: 216 шт., Объем: 10,027,800,000 ₽, Цена: 4642.50000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:27:00+00:00, Кол-во лотов: 265 шт., Объем: 12,301,300,000 ₽, Цена: 4642.00000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:28:00+00:00, Кол-во лотов: 62 шт., Объем: 2,878,660,000 ₽, Цена: 4643.00000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:29:00+00:00, Кол-во лотов: 904 шт., Объем: 41,963,680,000 ₽, Цена: 4642.00000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:30:00+00:00, Кол-во лотов: 316 шт., Объем: 14,671,880,000 ₽, Цена: 4643.00000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:31:00+00:00, Кол-во лотов: 96 шт., Объем: 4,457,280,000 ₽, Цена: 4643.00000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:32:00+00:00, Кол-во лотов: 103 шт., Объем: 4,781,770,000 ₽, Цена: 4642.50000000 ₽
Лукойл (LKOH), Время: 2023-04-10 11:33:00+00:00, Кол-

In [23]:
def get_final_stock_volumes(_input):
    return f'Объем: {_input:,} ₽'

def get_final_lots(_lots):
    return f'Кол-во лотов: {_lots:,} шт.'


price = 4666.00000000
lots = 1085
print(f'Цена: {price} ₽')
print(get_final_lots(lots))
output = int(price * lots)
print(output)
print(get_final_stock_volumes(output))


Цена: 4666.0 ₽
Кол-во лотов: 1,085 шт.
5062610
Объем: 5,062,610 ₽
